<a href="https://colab.research.google.com/github/igorgcgv/igorgcgv/blob/main/CNPJ_de_Empresas_Gerando_Dados_Valiosos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Importando as Libs


import pandas as pd
import time
import json

In [2]:
data = "/content/CE_Bebedouro_Armarios_RFB.xlsx"

In [3]:
dataset = pd.read_excel (data)

In [ ]:
dataset.tail()

In [ ]:
dataset.columns

In [6]:
lista_drop_colunas = ['NOME CONTATO', 'NOME OPORTUNIDADE', 'NOME EMPRESA',
       'END-CIDADE', 'END-ESTADO', 'EMAIL', 'CONTATO', 'FONTE']

In [7]:
dataset = dataset.drop(lista_drop_colunas, axis= 1)

In [ ]:
dataset.head()

In [9]:
#Funcao converter STR
def convert_to_string(df, columns):
    for col in columns:
        df[col] = df[col].astype("string")

In [10]:
convert_to_string(dataset,['CNPJ'])

In [11]:
#Funcao para tratar CNPJ que começam com Zero
def colocar_zero (cnpj):
    if len(cnpj) == 13:
      return "0"+cnpj
    else:
      return cnpj

In [12]:
#Aplicando a funcao colocar_zero na coluna CNPJ

dataset['CNPJ_'] = dataset['CNPJ'].apply(lambda x: colocar_zero(x))

In [ ]:
dataset.head()

In [14]:
dataset = dataset.drop(['CNPJ'], axis = 1)

In [15]:
dataset = dataset.rename(columns={'CNPJ_':'CNPJ'})

In [ ]:
dataset.head()

In [25]:
from pandas.io.json import json_normalize
#########################
#Consumindo a API

import http.client
import requests

def consulta(cnpj):

  conn = http.client.HTTPSConnection("receitaws.com.br")

  headers = { 'Accept': "application/json" }

  conn.request("GET", f"/v1/cnpj/{cnpj}", headers=headers)

  res = conn.getresponse()
  data = json.loads(res.read())
  #df = json_normalize(data['nome'])

  #print(df)

  return data['nome'],data['logradouro'],data['numero'],data['cnpj'],data['bairro'],data['municipio'],data['uf'],data['cep'],data['telefone'],data['email'],data['atividade_principal']

In [ ]:
resultados = []

for i, row in dataset.head(2).iterrows():#o head é pra definir a quantidades de linhas que vc deseja interar (Deve ser excluido para poder interar em todas as linhas da coluna CNPJ)
    cnpj = row['CNPJ']
    resultado = consulta(cnpj)
    resultados.append(list(resultado))
    time.sleep(20)

df = pd.DataFrame(resultados, columns=['nome', 'logradouro', 'numero', 'cnpj', 'bairro', 'municipio', 'uf', 'cep', 'telefone', 'email','atividade_principal'])
print(df)

In [ ]:
#Verificando o DF
df

In [ ]:
type(df)

In [29]:
df.to_csv('empresas.csv', index=False)

In [30]:
###FIM####